In [ ]:
!pip install torch lightning torchvision pyav

In [2]:
# Training hyperparameters
IMG_SIZE = 112
FRAMES_PER_VIDEO = 8
NUM_CLASSES = 30
LEARNING_RATE = 0.001
BATCH_SIZE = 4
MAX_EPOCHS = 1000
MAX_TIME = "00:11:00:00"

# Dataset
DATA_DIR = "./datasets/key_clf_data_112_112"
NUM_WORKERS = 4

FAST_DEV_RUN = True
CHECKPOINT_DIR = "restnet/"

# Compute related
ACCELERATOR = "cpu"
DEVICES = 0

# Resnet 3D

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
from functools import partial


def conv3x3x3(in_planes, out_planes, stride=1):
    # 3x3x3 convolution with padding
    return nn.Conv3d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=1,
        bias=False)


def downsample_basic_block(x, planes, stride):
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.Tensor(
        out.size(0), 
        planes - out.size(1), 
        out.size(2), 
        out.size(3),
        out.size(4)
    ).zero_()
    
    if isinstance(out.data, torch.cuda.FloatStorage):
        zero_pads = zero_pads.cuda()

    out = Variable(torch.cat([out.data, zero_pads], dim=1))

    return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)

        self.conv2 = nn.Conv3d(
            planes, planes, 
            kernel_size=3, stride=stride, padding=1, 
            bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        
        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self,
                 block,
                 layers,
                 sample_size,
                 sample_duration,
                 shortcut_type='B',
                 num_classes=400):
        
        """
        block: basic block or bottle neck
        layers: define Resnet architecture 34, 101, 152 etc
        sample size: image size
        shortcut_type: 'A' or 'B'
        num_classes: ...
        """
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv3d(
            3,
            64,
            kernel_size=7,
            stride=(1, 2, 2),
            padding=(3, 3, 3),
            bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)


        self.layer1 = self._make_layer(
            block, 64, layers[0], shortcut_type)
        self.layer2 = self._make_layer(
            block, 128, layers[1], shortcut_type, stride=2)
        self.layer3 = self._make_layer(
            block, 256, layers[2], shortcut_type, stride=2)
        self.layer4 = self._make_layer(
            block, 512, layers[3], shortcut_type, stride=2)
        
        
        last_duration = int(math.ceil(sample_duration / 16))
        last_size = int(math.ceil(sample_size / 32))
        self.avgpool = nn.AvgPool3d(
            (last_duration, last_size, last_size), stride=1)
        
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(
                    downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride)
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(
                        self.inplanes,
                        planes * block.expansion,
                        kernel_size=1,
                        stride=stride,
                        bias=False), nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def get_fine_tuning_parameters(model, ft_portion):
    if ft_portion == "complete":
        return model.parameters()

    elif ft_portion == "last_layer":
        ft_module_names = []
        ft_module_names.append('classifier')

        parameters = []
        for k, v in model.named_parameters():
            for ft_module in ft_module_names:
                if ft_module in k:
                    parameters.append({'params': v})
                    break
            else:
                parameters.append({'params': v, 'lr': 0.0})
        return parameters
    else:
        raise ValueError("Unsupported ft_portion: 'complete' or 'last_layer' expected")


def resnet10(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [1, 1, 1, 1], **kwargs)
    return model


def resnet18(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model


def resnet34(**kwargs):
    """Constructs a ResNet-34 model.
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    return model


def resnet50(**kwargs):
    """Constructs a ResNet-50 model.
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model


def resnet101(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    return model


def resnet152(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    return model


def resnet200(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 24, 36, 3], **kwargs)
    return model


# Dataset

In [19]:
import pathlib
import torch
# from GesRec.models.resnet import resnet101
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_video
import lightning as L
from lightning.pytorch.loggers import CSVLogger
import torchmetrics
from lightning.pytorch.callbacks import EarlyStopping

class KeyClf(L.LightningModule):
    def __init__(self, img_size, frames_per_video, num_classes, learning_rate):
        super().__init__()
        self.model = resnet101(sample_size=img_size,
                 sample_duration=frames_per_video,
                 shortcut_type='A',
                 num_classes=num_classes)
        
        self.loss_fn = torch.nn.CrossEntropyLoss(torch.tensor(
            [0.3786, 3.4687, 0.2339, 5.1575, 0.5353, 2.7603, 1.2979, 1.3481, 0.3588,
            2.4195, 2.2690, 1.3504, 0.5582, 4.5445, 5.3329, 0.9848, 1.8958, 0.6644,
            0.6276, 1.7267, 5.6196, 0.6950, 0.8535, 0.5532, 1.1135, 4.1810, 2.9921,
             3.6377, 2.1390, 4.0409]))
        self.accuracy = torchmetrics.Accuracy(
            task="multiclass", num_classes=num_classes
        )

        self.training_step_outputs = []
        self.validate_step_outputs = []

        self.lr = learning_rate


    def training_step(self, batch):
        videos, targets = batch
        preds = self.model(videos)
        loss = self.loss_fn(preds, targets.long())
        self.training_step_outputs.append({"preds": preds, "targets": targets})

        return loss
    
    def on_training_epoch_end(self):
        preds = torch.cat(outputs["preds"] for outputs in self.training_step_outputs)
        targets = torch.cat(outputs["targets"] for outputs in self.training_step_outputs)
       
        self.log_dict(
            {
                "train_acc": self.accuracy(preds, targets),
            },
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )


        self.training_step_outputs.clear()


    def validation_step(self, batch):
        videos, targets = batch
        preds = self.model(videos)
        loss = self.loss_fn(preds, targets.long())
        self.validate_step_outputs.append({"preds": preds, "targets": targets})

        return loss
    
    
    def on_validation_epoch_end(self):
        preds = torch.cat(outputs["preds"] for outputs in self.validate_step_outputs)
        targets = torch.cat(outputs["targets"] for outputs in self.validate_step_outputs)
       
        self.log_dict(
            {
                "val_acc": self.accuracy(preds, targets),
                "val_loss": 
            },
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )

        self.validate_step_outputs.clear()
    
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.lr)
    
class KeyStrokeClsDataset(Dataset):
    def __init__(self, data_dir, mode):
        self.dataset_root_path = pathlib.Path(data_dir)
        self.all_video_file_paths =  list(self.dataset_root_path.glob(f"{mode}/*/*.mp4"))

        self.class_labels = sorted({str(path).split("/")[-2] for path in self.all_video_file_paths})
        
        self.label2id = {label: i for i, label in enumerate(self.class_labels)}
        self.id2label  = {i: label for label, i in self.label2id.items()}
  
    def __len__(self):
        return len(self.all_video_file_paths)

    def __getitem__(self, idx):
        file_path = self.all_video_file_paths[idx]
        vframes, _, _ = read_video(file_path, pts_unit='sec')
        label = str(file_path).split("/")[-2]

        # permute to (num_frames, num_channels, height, width)
        vframes = vframes.permute(3, 0, 1, 2).float() / 255.0
    
        return vframes, self.label2id[label]


class KeyClsData(L.LightningDataModule):
    def __init__(self, batch_size, data_dir):
        super().__init__()
        self.batch_size = batch_size
        self.dataset_root_path = pathlib.Path(data_dir)
        self.data_dir = data_dir
   
        
    def train_dataloader(self):
        train_dataset = KeyStrokeClsDataset(self.data_dir, 'train')
        print("Train dataset:", len(train_dataset))
        return DataLoader(train_dataset, batch_size=self.batch_size, num_workers=NUM_WORKERS, persistent_workers=True)
    
    def val_dataloader(self):
        val_dataset = KeyStrokeClsDataset(self.data_dir, 'val')
        print("Val dataset:", len(val_dataset))
        return DataLoader(val_dataset, batch_size=self.batch_size, num_workers=NUM_WORKERS, persistent_workers=True)
    
    def test_dataloader(self):
        test_dataset = KeyStrokeClsDataset(self.data_dir, 'test')
        print("Test dataset:", len(test_dataset))
        return DataLoader(test_dataset, batch_size=self.batch_size, num_workers=NUM_WORKERS, persistent_workers=True)

In [20]:
logger = CSVLogger("logs", name="restnet_test")
data = KeyClsData(batch_size=BATCH_SIZE, data_dir=DATA_DIR)
model = KeyClf(img_size=IMG_SIZE, 
            frames_per_video=FRAMES_PER_VIDEO,
            num_classes=NUM_CLASSES, 
            learning_rate=LEARNING_RATE,)

logger = CSVLogger("logs", name=f"resnet")
trainer = L.Trainer(
    deterministic=True,
    # devices=DEVICES,
    max_time=MAX_TIME,
    callbacks=[EarlyStopping(monitor="val_loss")],
    default_root_dir=CHECKPOINT_DIR,
    fast_dev_run=FAST_DEV_RUN,
    logger=logger,
    accelerator=ACCELERATOR
)
trainer.fit(model, data)

/var/folders/9w/wb24vz290fx3x35b66j6pds80000gn/T/ipykernel_51145/562908493.py:164: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/haily/.pyenv/versions/3.10.4/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.

  | Name     | Type               | Params | Mode 
--------------------------------------------------------
0 | model    | ResNet             | 82.5 M | train
1 | loss_fn  | CrossEntropyLoss   | 0      | train
2 | accuracy | MulticlassAccuracy | 0      | train
--------------------------------------------------------
82.5 M    Trainable pa

Train dataset: 47036


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/haily/.pyenv/versions/3.10.4/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/haily/.pyenv/versions/3.10.4/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'KeyStrokeClsDataset' on <module '__main__' (built-in)>
/Users/haily/.pyenv/versions/3.10.4/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


**TEST**

In [ ]:
from tqdm import tqdm
test_dataset = KeyStrokeClsDataset("/kaggle/input/keystroker-classify/keystroke_classify_data", 'test')

id2Label = ['BackSpace', 'Comma', 'Space', 'Stop', 
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 
            'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 
            'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
            'y', 'z']

model = KeyClf.load_from_checkpoint('/kaggle/working/clf_train_batch_8_max_epochs_10')
model.eval()
y = []
predictions = []
with torch.no_grad():
    for video, label in tqdm(test_dataset):
        logits = model.model(video.unsqueeze(0))
        y_pred = id2Label[torch.argmax(logits, dim=1).item()]
        predictions.append(y_pred)
        y.append(id2Label[label])
        # print(f"y_pred: {y_pred}; y: {id2Label[label]}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y, predictions))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# Create confusion matrix
cm = confusion_matrix(y, predictions)

# Plot confusion matrix
plt.figure(figsize=(16, 12))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=id2Label, yticklabels=id2Label)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


In [18]:
train_dataset = KeyStrokeClsDataset(DATA_DIR, 'train')
len(train_dataset)

47036